# 🔍 Aniga3 Chapter Processor - Colab Runner
## Setup and Run
1. Chạy bước 1 để tải model và môi trường.
2. Chạy bước 2 để bật Server.
Đừng quên Mount Google Drive để đọc dữ liệu từ Aniga Chapter Manager.

In [ ]:
# @title 1. Cài đặt Môi trường & Tải Model
import os
import requests
from google.colab import drive

# 1. Kết nối Drive
drive.mount('/content/drive')

# 2. Clone Git
!git clone https://github.com/jofix2004/Aniga-Pipeline.git /content/Aniga-Pipeline
os.chdir('/content/Aniga-Pipeline')
!git fetch origin master
!git reset --hard origin/master

# 3. Cài đặt thư viện
print("\nBƯỚC 2: Đang cài đặt các thư viện...")
%pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
os.chdir('/content/Aniga-Pipeline/aniga_aniga3')
%pip install -q -r requirements.txt

# 4. Tải Model
models_to_download = {
    "model1.pt": "https://huggingface.co/TranLinh2004/AniGa-CleMove/resolve/main/v8-m3.pt?download=true",
    "model2.pt": "https://huggingface.co/TranLinh2004/AniGa-CleMove/resolve/main/v9-c2_fp16.pt?download=true",
    "model3.pt": "https://huggingface.co/TranLinh2004/AniGa-CleMove/resolve/main/v11-m3.pt?download=true"
}

import requests
from tqdm.notebook import tqdm

def download_file(url, destination):
    if os.path.exists(destination): return True
    with requests.get(url, stream=True, timeout=60) as r:
        r.raise_for_status()
        total_size = int(r.headers.get('content-length', 0))
        with open(destination, 'wb') as f, tqdm(total=total_size, unit='iB', unit_scale=True) as bar:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
                bar.update(len(chunk))
    return True

for filename, url in models_to_download.items():
    download_file(url, os.path.join('/content/Aniga-Pipeline/aniga_aniga3', filename))

print("✅ Môi trường đã sẵn sàng!")


In [ ]:
# @title 2. Khởi động Aniga3 Server & Cloudflare Tunnel
import os
import sys
import nest_asyncio
import uvicorn
import threading
import subprocess
import time
import re

nest_asyncio.apply()

os.chdir('/content/Aniga-Pipeline/aniga_aniga3')
sys.path.append('/content/Aniga-Pipeline/aniga_aniga3')

# Import server app
import aniga3_server

print("🚀 Khởi động Aniga3 Chapter Processor (Port 8002)...")

# Start Server in Thread
server_config = uvicorn.Config(aniga3_server.app, host="0.0.0.0", port=8002, log_level="error")
server = uvicorn.Server(server_config)
t = threading.Thread(target=server.run, daemon=True)
t.start()
time.sleep(3)

# Cloudflared Tunnel
print("🌐 Thiết lập Cloudflare Tunnel...")
if not os.path.exists("/usr/bin/cloudflared"):
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
    !dpkg -i cloudflared-linux-amd64.deb
    os.remove("cloudflared-linux-amd64.deb")

cmd = "cloudflared tunnel --url http://localhost:8002"
process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

print("="*50)
print("👇👇👇 LINK TRUY CẬP ANIGA3: 👇👇👇")
try:
    for line in iter(process.stdout.readline, ''):
        if "trycloudflare.com" in line:
            match = re.search(r"https://[a-zA-Z0-9-]+\\.trycloudflare\\.com", line)
            if match:
                print(f"\n🔗 URL: {match.group(0)}\n")
                print("="*50)
                break
except KeyboardInterrupt:
    process.kill()
    print("\n🛑 Tunnel stopped.")
